### Authors: Tatiana Likhomanenko (contact), Alexey Rogozhnikov, Denis Derkach

In [1]:
from IPython.display import Image
import pandas

# Opposite side tagging (OS) tagging (Current version)

https://github.com/tata-antares/tagging_LHCb/blob/master/old-tagging.ipynb

### Electron, muon, kaon and vertex taggers

In [2]:
pandas.read_csv('img/old-tagging-parts.csv').drop(['AUC, with untag', '$\Delta$ AUC, with untag'], axis=1)

,name,"$\epsilon_{tag}, \%$","$\Delta \epsilon_{tag}, \%$",$D^2$,$\Delta D^2$,"$\epsilon, \%$","$\Delta \epsilon, \%$"
0,vtx_xgboost,18.197761,0.049494,0.051501,0.001068,0.937198,0.019601
1,vtx_tmva,18.197761,0.049494,0.041675,0.001045,0.758385,0.019133
2,$K$_xgboost,17.079611,0.047949,0.054600,0.001024,0.932552,0.017682
3,$K$_tmva,17.079611,0.047949,0.047581,0.001298,0.812670,0.022288
4,$e$_xgboost,1.642418,0.014869,0.174040,0.005515,0.285847,0.009421
5,$e$_tmva,1.642418,0.014869,0.173899,0.005989,0.285615,0.010170
6,$\mu$_xgboost,5.438510,0.027057,0.170515,0.004334,0.927348,0.024016
7,$\mu$_tmva,5.438510,0.027057,0.164109,0.003794,0.892508,0.021105


### Combined taggers

In [3]:
pandas.read_csv('img/old-tagging.csv').drop(['$\Delta$ AUC, with untag'], axis=1)

,name,"$\epsilon_{tag}, \%$","$\Delta \epsilon_{tag}, \%$",$D^2$,$\Delta D^2$,"$\epsilon, \%$","$\Delta \epsilon, \%$","AUC, with untag"
0,iso-xgb_combined,32.734997,0.066382,0.069744,0.001793,2.283070,0.058863,56.702260
1,iso-tmva_combined,32.734997,0.066382,0.066827,0.002069,2.187571,0.067886,56.584572
2,log-xgb_combined,32.734997,0.066382,0.074225,0.000979,2.429755,0.032420,56.668568
3,log-tmva_combined,32.734997,0.066382,0.067781,0.000781,2.218803,0.025944,56.587753


# Opposite side tagging (OS) tagging (NEW)

## Data: 

* real data $B^+ \to J/\psi K^+$, $B^- \to J/\psi K^-$ 
* apply sPlot to obtain sWeight ~ P(B)
* set `event_id` - EventNumber & RunNumber

## Tracking inclusive OS tagging
https://github.com/tata-antares/tagging_LHCb/blob/master/track-based-tagging.ipynb


### Selections

* Input: all possible tracks for all B-events.
* (PIDNNp < tr) & (PIDNNpi < tr) & (ghostProb < 0.4), tr=0.6
* ((PIDNNk > trk) | (PIDNNm > trm) | (PIDNNe > tre)), trk=0.7, trm=0.4, tre=0.6



### $\epsilon_{tag}$ calculation

$$N (\text{passed selection}) = \sum_{\text{passed selection}} sw_i$$

$$N (\text{all events}) = \sum_{\text{all events}} sw_i,$$

where $sw_i$ - sPLot weight

$$\epsilon_{tag} = \frac{N (\text{passed selection})} {N (\text{all events})}$$

$$\Delta\epsilon_{tag} = \frac{\sqrt{N (\text{passed selection})}} {N (\text{all events})}$$


### Data for training

* `data_sw_passed` - tracks with **B-sWeight > 1**, are used for training
* `data_sw_not_passed` - tracks with **B-sWeight <= 1**, are tagged after training

### B mass before sWeight cut

In [4]:
Image(url='https://raw.githubusercontent.com/tata-antares/tagging_LHCb/master/img/Bmass.png')

### B mass after sWeight cut

In [5]:
Image(url='https://raw.githubusercontent.com/tata-antares/tagging_LHCb/master/img/Bmass_selected.png')

### Number of tracks for events

In [6]:
Image(url='https://raw.githubusercontent.com/tata-antares/tagging_LHCb/master/img/tracks_number.png')

In [7]:
Image(url='https://raw.githubusercontent.com/tata-antares/tagging_LHCb/master/img/PID_selected.png')

### PIDNN distributions after selection

### Training 

#### Features (sig = signal, part = tagger track):

* `cos_diff_phi` = $\cos(\phi^{sig} - \phi^{\rm part})$
* `diff_pt` = $\max(p_T)^{part} - p_T(B^{sig})$
* `partPt`= $p_T^{part}$
* `max_PID_e_mu` = $\max(PIDNN(e), PIDNN(\mu))^{part}$
* `partP` = $p^{part}$
* `nnkrec` = Number of reconstructed vertices
* `diff_eta` = $(\eta^{sig} - \eta^{\rm part})$
* `EOverP` = E/P (from CALO)
* `sum_PID_k_mu` = $\sum\limits_{i\in part}(PIDNN(K)+PIDNN(\mu))$
* `ptB` = $p_T^{sig}$
* `sum_PID_e_mu` = $\sum\limits_{i\in part}(PIDNN(e)+PIDNN(\mu))$
* `sum_PID_k_e` = $\sum\limits_{i\in part}(PIDNN(K)+PIDNN(e))$
* `proj` = $(\vec{p}^{sig},\vec{p}^{part})$
* `PIDNNe` = $PIDNN(e)$
* `PIDNNk` = $PIDNN(K)$
* `PIDNNm` = $PIDNN(\mu)$
* `phi` = $\phi^{sig}$
* `IP` = number of IPs in the event
* `max_PID_k_mu` = $max(PIDNN(K)+PIDNN(\mu))$
* `IPerr` = error of IP
* `IPs` = IP/IPerr
* `ID` = ID of the track (depends on the track container, where the track comes from)
* `veloch` = dE/dx track charge from the VELO system
* `max_PID_k_e`  = $max(PIDNN(K)+PIDNN(e))$
* `diff_phi`  = $(\phi^{sig} - \phi^{\rm part})$
* `ghostProb` = ghost probability 
* `IPPU` =  impact parameter with respect to any other reconstructed primary vertex.
* `eta` = pseudorapity of signal particle
* `partlcs` = chi2PerDoF for a track 

#### Classifier

Try to define B sign using track sign (to define they have the same signs or opposite).

`target` = `signB` * `signTrack` > 0

* classifier returns $P(\text{track same sign as B| B sign}) = P(\text{B same sign as track| track sign})$
* 2-folding training on the full training sample to use full sample for futher analysis (folding scheme provides not overfitted model, details: http://yandex.github.io/rep/metaml.html#module-rep.metaml.folding)

#### Calibration of $P(\text{track same sign as B| B sign})$

* use 2-folding logistic calibration for track classifier's prediction
* compare with isotonic calibration (bad)
* compare with absent calibration (bad, have shift predictions)


#### Computation of $p(B^+)$ using $P(\text{track same sign as B| B sign})$

Compute $p(B^+)$ using this probabilistic model representation (similar to previous tagging combination):

$$ \frac{P(B^+)}{P(B^-)} = \prod_{track, vertex} \frac{P(\text{track/vertex}|B^+)} {P(\text{track/vertex} |B^-)} = P
\qquad \Rightarrow\qquad P(B^+) = \frac {P}{1+P},   \qquad \qquad    [1] $$
where

$$
\frac{P(B^+)}{P(B^-)} =  
\begin{cases}
\frac{P(\text{track/vertex same sign as } B| B)}{P(\text{track/vertex opposite sign as } B| B)}, \text{if track/vertex}^+ \\ \\
\frac{P(\text{track/vertex opposite sign as } B| B)}{P(\text{track/vertex same sign as } B| B)}, \text{if track/vertex}^- 
\end{cases}
$$

#### Intermediate estimation $<D^2>$ for tracking

Do calibration of $p(B^+)$ and compute $<D^2>$ :

* use Isotonic calibration (generalization of bins fitting by linear function) - piecewise-constant monotonic function
* randomly divide events into two parts (1-train, 2-calibrate)
* Isotonic fitting on train and $<D^2>$ computation on test
* take mean and std for computed $<D^2>$

$<D^2>$  formula for sample:
$$<D^2> =  \frac{\sum_i[2(p^{mistag}_i - 0.5)]^2 * sw_i}{\sum_i sw_i}
=  \frac{\sum_i[2(p_i(B^+) - 0.5)]^2 * sw_i}{\sum_i sw_i}$$

Formula is symmetric and it is not necessary to compute mistag probability

## Vertex OS tagging
https://github.com/tata-antares/tagging_LHCb/blob/master/vertex-based-tagging.ipynb


### Selections

* All selection in c++ code are removed except DaVinci probability cuts


### Data for training

* `data_sw_passed` - tracks with **B-sWeight > 1**, are used for training
* `data_sw_not_passed` - tracks with **B-sWeight <= 1**, are tagged after training

### Training 

#### Features:

* `mult` = multiplicity in the event
* `nnkrec` = number of reconstructed vertices 
* `ptB` = signal B transverse momentum 
* `vflag` = number of tracks in the vertex
* `ipsmean` = mean distance to IP of the tracks
* `ptmean`  = mean pt of the tracks
* `vcharge` = charge of the vertex weigthed by pt
* `svm` = mass of the vertex 
* `svp` = momentum of the vertex
* `BDphiDir` = distance betwen B and vertex direction
* `svtau`  = lifetime of the vertex
* `docamax` = mean DOCA of the tracks
             
#### Classifier

Try to define B sign using vertex sign (to define they have the same signs or opposite).

`target` = `signB` * `signTrack` > 0

* classifier returns $P(\text{vertex same sign as B| B sign}) = P(\text{B same sign as vertex| vertex sign})$
* 2-folding training on the full training sample to use full sample for futher analysis (folding scheme provides not overfitted model, details: http://yandex.github.io/rep/metaml.html#module-rep.metaml.folding)

#### Calibration of $P(\text{vertex same sign as B| B sign})$

* use 2-folding isotonic calibration for vertex classifier's prediction
* compare with isotonic calibration (bad)

#### Computation of $p(B^+)$ using $P(\text{vertex same sign as B| B sign})$

The same formula as for tracks.

#### Intermediate estimation $<D^2>$ for vertices

Do calibration of $p(B^+)$ and compute $<D^2>$ :

* use Isotonic calibration (generalization of bins fitting by linear function) - piecewise-constant monotonic function
* randomly divide events into two parts (1-train, 2-calibrate)
* Isotonic fitting on train and $<D^2>$ computation on test
* take mean and std for computed $<D^2>$

$<D^2>$  formula for sample:
$$<D^2> =  \frac{\sum_i[2(p^{mistag}_i - 0.5)]^2 * sw_i}{\sum_i sw_i}
=  \frac{\sum_i[2(p_i(B^+) - 0.5)]^2 * sw_i}{\sum_i sw_i}$$

Formula is symmetric and it is not necessary to compute mistag probability

## Preliminary estimation

### $\epsilon$ calculation

$$\epsilon = <D^2> * \epsilon_{tag}$$
$$\Delta \epsilon = \sqrt{ \left(\frac{\Delta <D^2>}{<D^2>}\right)^2 + \left(\frac{\Delta \epsilon_{tag} }{\epsilon_{tag}} \right)^2 }$$

* Combine track-based and vertex-based tagging using formula [1]
* Isotonic calibration on random subsample with $D^2$ calculation
* take mean and std for computed $<D^2>$

https://github.com/tata-antares/tagging_LHCb/blob/master/combined-tagging.ipynb

In [8]:
pandas.read_csv('img/new-tagging.csv').drop(['$\Delta$ AUC, with untag'], axis=1)

,name,"$\epsilon_{tag}, \%$","$\Delta \epsilon_{tag}, \%$",$D^2$,$\Delta D^2$,"$\epsilon, \%$","$\Delta \epsilon, \%$","AUC, with untag"
0,Inclusive tagging,77.789955,0.102331,0.034991,0.000497,2.72191,0.038805,57.951077


## Full estimation of systematic error

https://github.com/tata-antares/tagging_LHCb/blob/master/tagging-full-systematic.ipynb

* set random state
* train the best model (track and vertex taggers with 2-folding with fixed random state)
* do calibration for track and vertex taggers with 2-folding with fixed random state
* compute $p(B^+)$
* do calibration with isotonic 2-folding (random state is fixed)
* compute $<D^2>$

This procedure is repeated (from the scratch) for 30 different random states and then we compute **mean** and **std** for these 30 values of $<D^2>$.

In [9]:
pandas.read_csv('img/new-tagging-systematic.csv')

,name,"$\epsilon_{tag}, \%$","$\Delta \epsilon_{tag}, \%$",$D^2$,$\Delta D^2$,"$\epsilon, \%$","$\Delta \epsilon, \%$","AUC, with untag","$\Delta$ AUC, with untag"
0,NEW + full systematic,77.789955,0.102331,0.034918,0.000103,2.71629,0.008774,57.958599,0.02456


## Calibration checking

In [10]:
Image(url='https://raw.githubusercontent.com/tata-antares/tagging_LHCb/master/img/Bprob_iso_calibrated.png')

### Isotonic transformation

In [11]:
Image(url='https://raw.githubusercontent.com/tata-antares/tagging_LHCb/master/img/iso_transformation.png')

### Check calibration of mistag
* axis x: predicted mistag probability 
$$p_{mistag} = min(p(B^+), p(B^-))$$
* axis y: true mistag probability (computed for bin)
$$p_{mistag} = \frac{N_{wrong}} {N_{wrong} + N_{right}}$$

$$\Delta p_{mistag} = \frac{\sqrt{N_{wrong} N_{right}}} {(N_{wrong} + N_{right})^{1.5}}$$

In [12]:
Image(url='https://raw.githubusercontent.com/tata-antares/tagging_LHCb/master/img/Bprob_calibration_check_uniform.png')

In [13]:
Image(url='https://raw.githubusercontent.com/tata-antares/tagging_LHCb/master/img/Bprob_calibration_check_percentile.png')

In [14]:
Image(url='https://raw.githubusercontent.com/tata-antares/tagging_LHCb/master/img/Bprob_calibration_check_iso_uniform.png')

In [15]:
Image(url='https://raw.githubusercontent.com/tata-antares/tagging_LHCb/master/img/Bprob_calibration_check_iso_percentile.png')

## TODO in progress

* convert python models to xml format (for production stage)
* add feature idea (Stephania)
* get all vertices from DaVinci